## Pruebas para generar las columnas de las matrices

In [5]:
import pyodbc
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

def get_sql_connection():
    """
    Establece la conexión con SQL Server.
    Configura los datos de conexión antes de ejecutar.
    """
    try:
        conn = pyodbc.connect(
            "DRIVER={ODBC Driver 17 for SQL Server};"
            f"SERVER={os.getenv('DB_SERVER')};"
            f"DATABASE={os.getenv('DATABASE')};"
            f"UID={os.getenv('DB_USER')};"
            f"PWD={os.getenv('DB_PASSWORD')}"
        )
        print("Conexión exitosa a la base de datos.")
        return conn
    except pyodbc.Error as e:
        print(f"Error al conectar a la base de datos: {str(e)}")
        raise

def execute_query(query):
    """
    Ejecuta el query en la base de datos y retorna un DataFrame.
    """
    conn = get_sql_connection()
    try:
        df = pd.read_sql(query, conn)
    except Exception as e:
        print(f"Error al ejecutar el query: {str(e)}")
        df = pd.DataFrame()  # Retorna un DataFrame vacío en caso de error
    finally:
        conn.close()
    return df

def get_inventario_matriz():
    """
    Extrae todos los registros de la tabla InventarioMatriz.
    """
    query = """
    SELECT 
        id_politica_base_riesgo,
        subsegmento,
        negocio,
        estado,
        cobertura
    FROM InventarioMatriz
    """
    return execute_query(query)

def get_matrices_base_riesgo():
    """
    Extrae todos los registros de la tabla MatricesBaseRiesgo.
    """
    query = """
    SELECT
        id_politica_base_riesgo,
        concatenado,
        segmento,
        permanencia,
        factor_prov,
        clasificacion,
        tipo_matriz
    FROM MatrizBaseRiesgo
    """
    return execute_query(query)

def unir_dataframes_merge():
    """
    Extrae los datos de ambas tablas, los unifica usando pd.merge() y convierte todos los campos a mayúsculas.
    """
    df_inventario = get_inventario_matriz()
    df_matrices = get_matrices_base_riesgo()
    
    # Se une utilizando la columna en común 'id_politica_base_riesgo'
    df_merge = pd.merge(df_inventario, df_matrices, 
                        on="id_politica_base_riesgo", 
                        how="inner")  # Cambia 'inner' por 'left' o 'outer' según lo requieras
    
    # Convertir todos los campos a mayúsculas
    df_merge = df_merge.applymap(lambda x: x.upper() if isinstance(x, str) else x)
    df_merge['factor_prov'] = df_merge['factor_prov'].apply(lambda x: x/100)
    
    
    return df_merge



In [6]:
df_merge = unir_dataframes_merge()
df_merge

Conexión exitosa a la base de datos.
Conexión exitosa a la base de datos.


C:\Users\prac.planeacionfi\AppData\Local\Temp\ipykernel_22992\2033789348.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\prac.planeacionfi\AppData\Local\Temp\ipykernel_22992\2033789348.py:86: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_merge = df_merge.applymap(lambda x: x.upper() if isinstance(x, str) else x)


,id_politica_base_riesgo,subsegmento,negocio,estado,cobertura,concatenado,segmento,permanencia,factor_prov,clasificacion,tipo_matriz
0,1,None,FPT,OBSOLETO,None,FPTOBSOLETO1.MENOR DE 90 DIAS,None,1.MENOR DE 90 DIAS,0.00,BAJO,MATRIZ NATURACO
1,2,None,FPT,OBSOLETO,None,FPTOBSOLETO2.ENTRE 90 Y 180 DIAS,None,2.ENTRE 90 Y 180 DIAS,0.15,MEDIO,MATRIZ NATURACO
2,3,None,FPT,OBSOLETO,None,FPTOBSOLETO3.ENTRE 180 Y 270 DIAS,None,3.ENTRE 180 Y 270 DIAS,0.30,MEDIO,MATRIZ NATURACO
3,4,None,FPT,OBSOLETO,None,FPTOBSOLETO4.ENTRE 270 Y 360 DIAS,None,4.ENTRE 270 Y 360 DIAS,0.50,MEDIO-ALTO,MATRIZ NATURACO
4,5,None,FPT,OBSOLETO,None,FPTOBSOLETO5.ENTRE 360 Y 540 DIAS,None,5.ENTRE 360 Y 540 DIAS,1.00,MUY ALTO,MATRIZ NATURACO
...,...,...,...,...,...,...,...,...,...,...,...
1150,1151,TOLL,None,None,5.MAYOR O IGUAL A 360 DIAS,EXPERTOS LOCALESTOLL5.MAYOR O IGUAL A 360 DIAS...,EXPERTOS LOCALES,3.ENTRE 180 Y 270 DIAS,0.80,MUY ALTO,PVA 4 A 6 MESES
1151,1152,TOLL,None,None,5.MAYOR O IGUAL A 360 DIAS,EXPERTOS LOCALESTOLL5.MAYOR O IGUAL A 360 DIAS...,EXPERTOS LOCALES,4.ENTRE 270 Y 360 DIAS,0.80,MUY ALTO,PVA 4 A 6 MESES
1152,1153,TOLL,None,None,5.MAYOR O IGUAL A 360 DIAS,EXPERTOS LOCALESTOLL5.MAYOR O IGUAL A 360 DIAS...,EXPERTOS LOCALES,5.ENTRE 360 Y 540 DIAS,0.80,MUY ALTO,PVA 4 A 6 MESES
1153,1154,TOLL,None,None,5.MAYOR O IGUAL A 360 DIAS,EXPERTOS LOCALESTOLL5.MAYOR O IGUAL A 360 DIAS...,EXPERTOS LOCALES,6.ENTRE 540 Y 720 DIAS,0.80,MUY ALTO,PVA 4 A 6 MESES
